# Key Levels Detection (Close)

In [23]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import seaborn as sns
import time
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import requests

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120

import warnings
warnings.filterwarnings('ignore')

In [24]:
entrade_headers = {
        'authority': 'services.entrade.com.vn',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'en-US,en;q=0.9',
        'dnt': '1',
        'origin': 'https://banggia.dnse.com.vn',
        'referer': 'https://banggia.dnse.com.vn/',
        'sec-ch-ua': '"Edge";v="114", "Chromium";v="114", "Not=A?Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'cross-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1788.0'
    }
DNSE_DATA_HISTORY_URL = 'https://services.entrade.com.vn/chart-api/v2/ohlcs/derivative'

In [25]:
def getStockHistoryData(ticker, timestamp_from=0, timestamp_to=0):
    if timestamp_from == 0:
        three_months = date.today() + relativedelta(months=-3)
        timestamp_from = datetime.strptime(three_months.strftime("%m/%d/%Y") + ', 00:00:0', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()
    if timestamp_to == 0:
        timestamp_to = datetime.strptime(date.today().strftime("%m/%d/%Y") + ', 23:59:00', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()

    params = {
        "resolution": "5",
        "symbol": "VN30F1M",
        "from": int(timestamp_from),
        "to": int(timestamp_to)
    }

    x = requests.get(DNSE_DATA_HISTORY_URL, params=params, headers=entrade_headers)
    response = x.json()

    import numpy as np
    import pandas as pd

    timestamp = np.array(response['t']).astype(int)
    close = np.array(response['c']).astype(float)
    open = np.array(response['o']).astype(float)
    high = np.array(response['h']).astype(float)
    low = np.array(response['l']).astype(float)
    volume = np.array(response['v']).astype(int)

    dataset = pd.DataFrame({'Time': timestamp, 'Open': list(open), 'High': list(high), 'Low': list(low),
                            'Close': list(close), 'Volume': list(volume)},
                           columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    return dataset
def prepareData(htd):
    if 'Time' in htd.columns:
        from datetime import datetime

        htd['DateStr'] = htd.apply(
            lambda x: datetime.fromtimestamp(x['Time']).strftime("%Y-%m-%d %H:%M:%S"), axis=1)

    htd['Date'] = pd.to_datetime(htd['DateStr'])
    # htd['Date'] = htd['Date'] + pd.DateOffset(hours=7)
    ticker_data = htd.set_index('Date')
    ticker_data.drop(columns=['Time', 'DateStr'], inplace=True)
    return ticker_data

## Load price data from DNSE

In [26]:
ticker = "VN30F1M"
htd = getStockHistoryData(ticker, 0, 0)
ticker_data = prepareData(htd)
dataset = ticker_data.dropna()
dataset

,Open,High,Low,Close,Volume
Date,,,,,
2024-07-23 09:00:00,1301.5,1302.7,1301.5,1302.2,4581
2024-07-23 09:05:00,1302.2,1302.4,1300.9,1301.9,2358
2024-07-23 09:10:00,1301.9,1302.1,1301.2,1301.8,1474
2024-07-23 09:15:00,1302.0,1303.6,1301.8,1303.3,3480
2024-07-23 09:20:00,1303.3,1303.7,1301.9,1302.0,3734
...,...,...,...,...,...
2024-10-23 14:15:00,1351.5,1353.5,1350.4,1351.2,9302
2024-10-23 14:20:00,1351.3,1355.4,1351.3,1353.2,13513
2024-10-23 14:25:00,1353.1,1353.2,1351.4,1353.2,7008


In [27]:
data = dataset.copy()
back_bar = 20
data['max_prev'] = data['High'].rolling(back_bar).max()
data['min_prev'] = data['Low'].rolling(back_bar).min()
data["RSI"] = ta.rsi(data["Close"], length=14)
data["ema"] = ta.ema(data["Close"], length=20)
data["diff_ema"] = data["ema"] - data["Close"]
data

,Open,High,Low,Close,Volume,max_prev,min_prev,RSI,ema,diff_ema
Date,,,,,,,,,,
2024-07-23 09:00:00,1301.5,1302.7,1301.5,1302.2,4581,NaN,NaN,NaN,NaN,NaN
2024-07-23 09:05:00,1302.2,1302.4,1300.9,1301.9,2358,NaN,NaN,NaN,NaN,NaN
2024-07-23 09:10:00,1301.9,1302.1,1301.2,1301.8,1474,NaN,NaN,NaN,NaN,NaN
2024-07-23 09:15:00,1302.0,1303.6,1301.8,1303.3,3480,NaN,NaN,NaN,NaN,NaN
2024-07-23 09:20:00,1303.3,1303.7,1301.9,1302.0,3734,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2024-10-23 14:15:00,1351.5,1353.5,1350.4,1351.2,9302,1353.9,1348.3,50.013123,1351.120217,-0.079783
2024-10-23 14:20:00,1351.3,1355.4,1351.3,1353.2,13513,1355.4,1348.3,56.913206,1351.318291,-1.881709
2024-10-23 14:25:00,1353.1,1353.2,1351.4,1353.2,7008,1355.4,1348.3,56.913206,1351.497502,-1.702498


In [28]:
data['is_r_keylevel'] = data.apply(lambda r: True if r['High'] == r['max_prev'] else False, axis=1)
data['is_s_keylevel'] = data.apply(lambda r: True if r['Low'] == r['min_prev'] else False, axis=1)
data

,Open,High,Low,Close,Volume,max_prev,min_prev,RSI,ema,diff_ema,is_r_keylevel,is_s_keylevel
Date,,,,,,,,,,,,
2024-07-23 09:00:00,1301.5,1302.7,1301.5,1302.2,4581,NaN,NaN,NaN,NaN,NaN,False,False
2024-07-23 09:05:00,1302.2,1302.4,1300.9,1301.9,2358,NaN,NaN,NaN,NaN,NaN,False,False
2024-07-23 09:10:00,1301.9,1302.1,1301.2,1301.8,1474,NaN,NaN,NaN,NaN,NaN,False,False
2024-07-23 09:15:00,1302.0,1303.6,1301.8,1303.3,3480,NaN,NaN,NaN,NaN,NaN,False,False
2024-07-23 09:20:00,1303.3,1303.7,1301.9,1302.0,3734,NaN,NaN,NaN,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-23 14:15:00,1351.5,1353.5,1350.4,1351.2,9302,1353.9,1348.3,50.013123,1351.120217,-0.079783,False,False
2024-10-23 14:20:00,1351.3,1355.4,1351.3,1353.2,13513,1355.4,1348.3,56.913206,1351.318291,-1.881709,True,False
2024-10-23 14:25:00,1353.1,1353.2,1351.4,1353.2,7008,1355.4,1348.3,56.913206,1351.497502,-1.702498,False,False


In [29]:
keylevel_data = data[(data['is_r_keylevel'] == True) | (data['is_s_keylevel'] == True)]
len(keylevel_data)

768

In [30]:
keylevel_data

,Open,High,Low,Close,Volume,max_prev,min_prev,RSI,ema,diff_ema,is_r_keylevel,is_s_keylevel
Date,,,,,,,,,,,,
2024-07-23 10:35:00,1299.3,1299.8,1298.6,1298.9,2291,1303.7,1298.6,34.446732,1301.120000,2.220000,False,True
2024-07-23 13:00:00,1302.0,1302.4,1301.4,1301.7,3698,1302.4,1298.6,55.793387,1300.826195,-0.873805,True,False
2024-07-23 13:30:00,1301.5,1301.6,1297.3,1298.2,10808,1302.4,1297.3,37.671492,1300.772856,2.572856,False,True
2024-07-23 13:35:00,1298.0,1298.5,1296.6,1297.9,7653,1302.4,1296.6,36.620487,1300.499251,2.599251,False,True
2024-07-23 13:40:00,1297.8,1298.0,1295.5,1296.0,6740,1302.4,1295.5,30.766101,1300.070751,4.070751,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-23 13:40:00,1352.0,1353.0,1351.8,1352.5,4853,1353.0,1346.2,57.410761,1351.100662,-1.399338,True,False
2024-10-23 13:45:00,1352.6,1353.9,1351.9,1353.9,5556,1353.9,1346.7,63.330434,1351.367266,-2.532734,True,False
2024-10-23 13:50:00,1353.8,1353.9,1350.8,1351.3,6363,1353.9,1346.7,49.554751,1351.360859,0.060859,True,False


In [31]:
keylevel_data[keylevel_data.index > '2024-10-23 08:40:00']

,Open,High,Low,Close,Volume,max_prev,min_prev,RSI,ema,diff_ema,is_r_keylevel,is_s_keylevel
Date,,,,,,,,,,,,
2024-10-23 10:15:00,1350.8,1351.2,1348.8,1349.2,6819,1357.0,1348.8,25.982963,1352.812651,3.612651,False,True
2024-10-23 10:20:00,1349.1,1350.1,1348.5,1350.1,3894,1354.4,1348.5,32.410946,1352.554303,2.454303,False,True
2024-10-23 10:25:00,1350.0,1350.1,1348.0,1348.5,5269,1354.4,1348.0,27.790339,1352.168179,3.668179,False,True
2024-10-23 10:30:00,1348.7,1348.7,1345.5,1347.0,8523,1354.4,1345.5,24.293657,1351.675972,4.675972,False,True
2024-10-23 10:35:00,1346.9,1347.1,1345.1,1346.8,6756,1354.4,1345.1,23.862533,1351.211593,4.411593,False,True
2024-10-23 13:00:00,1352.3,1352.5,1351.3,1351.5,3965,1352.5,1345.1,54.448128,1350.202855,-1.297145,True,False
2024-10-23 13:10:00,1352.0,1352.6,1351.5,1351.5,3252,1352.6,1345.1,53.903092,1350.472631,-1.027369,True,False
2024-10-23 13:20:00,1351.9,1353.0,1351.9,1352.3,4981,1353.0,1345.1,58.106503,1350.769660,-1.530340,True,False
2024-10-23 13:40:00,1352.0,1353.0,1351.8,1352.5,4853,1353.0,1346.2,57.410761,1351.100662,-1.399338,True,False
